In [3]:
# --- Setup / Requirements ---
!pip install selenium beautifulsoup4 pandas matplotlib numpy notebook

CHROMEDRIVER_PATH = r"C:\Users\DELL\Downloads\chromedriver-win64\chromedriver.exe"

HEADLESS = True  

print('Update CHROMEDRIVER_PATH before running the scraping cells.')


Update CHROMEDRIVER_PATH before running the scraping cells.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install webdriver-manager


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# --- Cars24 Mahindra Scraper (Mumbai, Pune, Hyderabad) ---

import time, random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Step 1: Setup Chrome
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Step 2: Target locations
cities = {
    "mumbai": "https://www.cars24.com/buy-used-mahindra-cars-mumbai/",
    "pune": "https://www.cars24.com/buy-used-mahindra-cars-pune/",
    "hyderabad": "https://www.cars24.com/buy-used-mahindra-cars-hyderabad/"
}

all_data = []

for city, url in cities.items():
    print(f"\nScraping Mahindra cars in {city.title()} ...")
    driver.get(url)
    time.sleep(5)

    # Step 3: Scroll until all cars load
    last_count = 0
    scroll_tries = 0
    while True:
        car_cards = driver.find_elements(By.CSS_SELECTOR, "a.styles_carCardWrapper__sXLIp")
        current_count = len(car_cards)

        if current_count > last_count:
            last_car = car_cards[-1]
            driver.execute_script("arguments[0].scrollIntoView();", last_car)
            time.sleep(random.uniform(3, 5))
            last_count = current_count
            scroll_tries = 0
            print(f"{city}: Loaded {current_count} cars")
        else:
            scroll_tries += 1
            if scroll_tries > 3:
                print(f"{city}: No more cars found.")
                break

    # Step 4: Parse HTML
    soup = BeautifulSoup(driver.page_source, "html.parser")

    def parse_car_card(card):
        data = {"city": city}

        # # Car name (model)
        # title = card.find("div", class_="sc-fLVwEd")
        # data["name"] = title.get_text(strip=True) if title else None

        # Specs
        specs = [p.get_text(strip=True) for p in card.select("ul p.sc-braxZu")]
        if len(specs) >= 4:
            data["kms"] = specs[0]
            data["fuel"] = specs[1]
            data["transmission"] = specs[2]
            data["year"] = specs[3]

        # Price
        price = card.find("p", class_="sc-braxZu cyPhJl")
        data["price"] = price.get_text(strip=True) if price else None

        # Location
        location = card.find("div", class_="styles_ellipsis__uatjG")
        data["location"] = location.get_text(strip=True) if location else None

        # Owners
        owners = card.find("p", string=lambda t: t and "Owner" in t)
        data["owners"] = owners.get_text(strip=True) if owners else None

        return data

    # Step 5: Collect data
    car_cards = soup.find_all("a", class_="styles_carCardWrapper__sXLIp")
    print(f"{city}: Total cars found: {len(car_cards)}")

    city_data = [parse_car_card(card) for card in car_cards]
    all_data.extend(city_data)

# Step 6: Save combined CSV
driver.quit()
df = pd.DataFrame(all_data)
df.to_csv("mahindra_cars_all_cities.csv", index=False, encoding="utf-8-sig")
print("\nCSV saved as mahindra_cars_all_cities.csv")

# Step 7: Preview first few rows
df.head()



Scraping Mahindra cars in Mumbai ...
mumbai: Loaded 20 cars
mumbai: Loaded 40 cars
mumbai: Loaded 60 cars
mumbai: Loaded 65 cars
mumbai: No more cars found.
mumbai: Total cars found: 65

Scraping Mahindra cars in Pune ...
pune: Loaded 14 cars
pune: No more cars found.
pune: Total cars found: 14

Scraping Mahindra cars in Hyderabad ...
hyderabad: Loaded 20 cars
hyderabad: Loaded 40 cars
hyderabad: Loaded 60 cars
hyderabad: Loaded 76 cars
hyderabad: No more cars found.
hyderabad: Total cars found: 76

CSV saved as mahindra_cars_all_cities.csv


,city,kms,fuel,transmission,year,price,location,owners
0,mumbai,21.21k km,Diesel,Manual,MH-46,₹8.16 lakh,Seawood,None
1,mumbai,71.84k km,Diesel,Manual,MH-05,₹6.35 lakh,"Korum Mall, Thane West",None
2,mumbai,91.63k km,Diesel,Auto,MH-04,₹5.80 lakh,"Goregaon, Mumbai",None
3,mumbai,11.34k km,Petrol,Manual,MH-17,₹9.16 lakh,Thakur Village Kandivali E Mumbai,None
4,mumbai,16.25k km,Diesel,Manual,MH-46,₹2.99 lakh,Panvel Navi Mumbai,None
